# Première classification

L'objectif ici est d'utiliser un premier classifier. Pour commencer nous prendrons SVM qui obtient souvent de bons résultats sur les données textuelles.

Nous pouvons donc, pour simplifier, créer un jeu d'apprentissage et un jeu de test et évaluer le résultat d'un classifieur SVM placé dans un pipeline.

In [60]:
from pretraitement import TextPreTraitement
from clean import clean_claimKG
from classification import trueVSfalse, trueFalseVSmixture, trueVSfalseVSmixture

import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix

from sklearn.neighbors import KNeighborsClassifier as KNN

In [2]:
file_name = "../data/claimKG.csv"
origin = pd.read_csv(file_name)

In [7]:
origin = clean_claimKG(origin, verbose=False, inplace=True)

kg = origin.copy()

Pour un premier test, nous allons prendre comme `X` les assertions et `Y` les valeurs de véracité.

Note: les valeurs de véracité ne sont pour l'instant pas disponible, à voir ou les trouver.

Donc, pour l'instant, on ne garde que les lignes qu'on peut exploiter.

In [48]:
kg = trueVSfalse(origin)
print(kg['ratingValue'].value_counts())

0    10701
1     4584
Name: ratingValue, dtype: int64


In [49]:
kg = cut_data(kg)
print(kg['ratingValue'].value_counts())

0    4584
1    4584
Name: ratingValue, dtype: int64


Création d'un jeu d'apprentissage et de test

In [56]:
X=kg['claimReview_claimReviewed']
y=kg['ratingValue']

X_train,X_test,y_train,y_test=train_test_split(X, y, 
                                               train_size=0.7 ,
                                               random_state=30,stratify=y)

In [51]:
vectorizer = TfidfVectorizer(
            lowercase=False,
            ngram_range=(1,2),
            tokenizer=None,
            preprocessor=lambda x:x,
            min_df=0.01, 
            max_df=0.9)

Création du pipeline

In [64]:
pipe = Pipeline([("cleaner", TextPreTraitement(stopword=True)),
                 ("count_vectorizer", vectorizer),
                 ("SVM", KNN(6))])
pipe.fit(X_train,y_train)

print("pipeline créé")

pipeline créé


On teste le modèle

In [65]:
y_pred = pipe.predict(X_test)

print("train accuracy:",accuracy_score(y_train, pipe.predict(X_train)))
print("test accuracy:",accuracy_score(y_test, y_pred))


train accuracy: 0.7026647966339411
test accuracy: 0.6441294074881861


In [66]:
confMatrix = confusion_matrix(y_test, y_pred)
print(confMatrix)

[[1195  180]
 [ 799  577]]
